In [1]:
import numpy as np
import scipy
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
from pynndescent import NNDescent

import time
import math
from sklearn.metrics.pairwise import KERNEL_PARAMS
import torch
import sys
import argparse
from tqdm import tqdm

from sklearn.metrics import pairwise_distances
from singleVis.data import DataProvider
from singleVis.backend import construct_spatial_temporal_complex, select_points_step
import singleVis.config as config
from singleVis.utils import hausdorff_dist_cus

In [2]:
DATASET = "cifar10"
CONTENT_PATH = "/home/xianglin/projects/DVI_data/TemporalExp/resnet18_{}".format(DATASET)

In [3]:
LEN = config.dataset_config[DATASET]["TRAINING_LEN"]
LAMBDA = config.dataset_config[DATASET]["LAMBDA"]
DOWNSAMPLING_RATE = config.dataset_config[DATASET]["DOWNSAMPLING_RATE"]
L_BOUND = config.dataset_config[DATASET]["L_BOUND"]

# define hyperparameters

DEVICE = torch.device("cuda:2")
EPOCH_NUMS = config.dataset_config[DATASET]["training_config"]["EPOCH_NUM"]
TIME_STEPS = config.dataset_config[DATASET]["training_config"]["TIME_STEPS"]
TEMPORAL_PERSISTENT = config.dataset_config[DATASET]["training_config"]["TEMPORAL_PERSISTENT"]
NUMS = config.dataset_config[DATASET]["training_config"]["NUMS"]    # how many epoch should we go through for one pass
PATIENT = config.dataset_config[DATASET]["training_config"]["PATIENT"]
TEMPORAL_EDGE_WEIGHT = config.dataset_config[DATASET]["training_config"]["TEMPORAL_EDGE_WEIGHT"]

content_path = CONTENT_PATH
sys.path.append(content_path)

from Model.model import *
net = resnet18()
classes = ("airplane", "car", "bird", "cat", "deer", "dog", "frog", "horse", "ship", "truck")
data_provider = DataProvider(content_path, net, 1, TIME_STEPS, 1, split=-1, device=DEVICE, verbose=1)

Finish initialization...


In [4]:
data = data_provider.train_representation(2)

In [5]:
from singleVis.kcenter_greedy import kCenterGreedy
from sklearn.metrics import pairwise_distances

# pairwise_distances(data, data)

idx = np.random.choice(np.arange(50000), 50, replace=False)
kc = kCenterGreedy(data[:50000])
_ = kc.select_batch_with_budgets(idx, 950)
idx = kc.already_selected

Calculating distances...
calculating distances for 50 points within 0.10 seconds...
Hausdorff distance is 1.56 with 1000 points


In [5]:
from singleVis.kcenter_greedy import kCenterGreedy
idx = np.random.choice(np.arange(50000), 50, replace=False)
kc = kCenterGreedy(data[:50000])
_ = kc.select_batch_with_budgets_s(idx, 950)
idx = kc.already_selected

Calculating distances...
calculating distances for 50 points within 0.10 seconds...
Hausdorff distance is 2.02 with 1000 points


In [9]:
from singleVis.apricot import FacilityLocationSelection
# X_pairwise = pairwise_distances(data, metric="euclidean", squared=True)
model = FacilityLocationSelection(1000, verbose=True, optimizer="lazy").fit(X_pairwise)

In [8]:
hausdorff_dist_cus(data, model.ranking)

Hausdorff distance 2.42 for 1000/50000 in 0.666 seconds...


(2.4152624139439607, 0.666)

In [5]:

import matplotlib
#matplotlib.use('TkAgg')

import heapq
import numpy as np
import pandas as pd
import scipy as sp
import math
from scipy import spatial
import matplotlib.pyplot as plt


class FacilityLocation:

    def __init__(self, D, V, alpha=1.):
        '''
        Args
        - D: np.array, shape [N, N], similarity matrix
        - V: list of int, indices of columns of D
        - alpha: float
        '''
        self.D = D
        self.curVal = 0
        self.curMax = np.zeros(len(D))
        self.gains = []
        self.alpha = alpha
        self.f_norm = self.alpha / self.f_norm(V)
        self.norm = 1. / self.inc(V, [])

    def f_norm(self, sset):
        return self.D[:, sset].max(axis=1).sum()

    def inc(self, sset, ndx):
        if len(sset + [ndx]) > 1:
            if not ndx:  # normalization
                return math.log(1 + self.alpha * 1)
            return self.norm * math.log(1 + self.f_norm * np.maximum(self.curMax, self.D[:, ndx]).sum()) - self.curVal
        else:
            return self.norm * math.log(1 + self.f_norm * self.D[:, ndx].sum()) - self.curVal

    def add(self, sset, ndx):
        cur_old = self.curVal
        if len(sset + [ndx]) > 1:
            self.curMax = np.maximum(self.curMax, self.D[:, ndx])
        else:
            self.curMax = self.D[:, ndx]
        self.curVal = self.norm * math.log(1 + self.f_norm * self.curMax.sum())
        self.gains.extend([self.curVal - cur_old])
        return self.curVal


def _heappush_max(heap, item):
    heap.append(item)
    heapq._siftdown_max(heap, 0, len(heap)-1)


def _heappop_max(heap):
    """Maxheap version of a heappop."""
    lastelt = heap.pop()  # raises appropriate IndexError if heap is empty
    if heap:
        returnitem = heap[0]
        heap[0] = lastelt
        heapq._siftup_max(heap, 0)
        return returnitem
    return lastelt


def lazy_greedy_heap(F, V, B):
    curVal = 0
    sset = []
    vals = []

    order = []
    heapq._heapify_max(order)
    [_heappush_max(order, (F.inc(sset, index), index)) for index in V]

    while order and len(sset) < B:
        el = _heappop_max(order)
        improv = F.inc(sset, el[1])

        # check for uniques elements
        if improv >= 0:
            if not order:
                curVal = F.add(sset, el[1])
                sset.append(el[1])
                vals.append(curVal)
            else:
                top = _heappop_max(order)
                if improv >= top[0]:
                    curVal = F.add(sset, el[1])
                    sset.append(el[1])
                    vals.append(curVal)
                else:
                    _heappush_max(order, (improv, el[1]))
                _heappush_max(order, top)

    return sset, vals


def test():
    n = 10
    X = np.random.rand(n, n)
    D = X * np.transpose(X)
    F = FacilityLocation(D, range(0, n))
    sset = lazy_greedy_heap(F, range(0, n), 15)
    print(sset)


In [6]:
X = pairwise_distances(data,data)

In [8]:
V = [i for i in range(50000)]
F = FacilityLocation(X, V)
B = 2000
sset, vals = lazy_greedy_heap(F, V, B)
print(len(sset))

2000


In [9]:
hausdorff_dist_cus(data, sset)

Hausdorff distance 2.79 for 2000/50000 in 1.127 seconds...


(2.7851256643713573, 1.127)

In [14]:
len(sset)

1000